<a href="https://colab.research.google.com/github/harshitadd/DP-NLP/blob/main/BERTNewsClassification_withoutDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet

     |████████████████████████████████| 2.3MB 7.5MB/s 
     |████████████████████████████████| 3.3MB 51.8MB/s 
     |████████████████████████████████| 901kB 51.1MB/s 


In [2]:
import zipfile
import urllib.request
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import transformers
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
from transformers.data.processors.utils import InputExample
from transformers.data.processors.glue import glue_convert_examples_to_features
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification


In [3]:
DATA_DIR = "/content/"

In [4]:
def download_and_extract(data_dir):
    print("Extracting Train zip...")
    filename = "train.csv.zip"
    with zipfile.ZipFile(filename) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(filename)
    print("Completed!")

    print("Extracting Test zip...")
    filename = "test.csv.zip"
    with zipfile.ZipFile(filename) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(filename)
    print("Completed!")

download_and_extract(DATA_DIR)

Extracting Train zip...
Completed!
Extracting Test zip...
Completed!


In [5]:
train_path =  '/content/train.csv'
dev_path = '/content/test.csv'

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(dev_path)

In [6]:
df_train = df_train.drop('Title', axis = 1)
df_test = df_test.drop('Title', axis = 1)
df_train = df_train.replace({'Class Index': {1: 'World', 2: 'Sports', 3: 'Business', 4: 'Sci/Tech'}})
df_test = df_test.replace({'Class Index': {1: 'World', 2: 'Sports', 3: 'Business', 4: 'Sci/Tech'}})

In [7]:
model_name = "bert-base-cased"
config = BertConfig.from_pretrained(
    model_name,
    num_labels=4,
)
tokenizer = BertTokenizer.from_pretrained(
    "bert-base-cased",
    do_lower_case=False,
)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased",
    config=config,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
# trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
trainable_layers = [model.classifier]
total_params = 0
trainable_params = 0

for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

for layer in trainable_layers:
    for p in layer.parameters():
        p.requires_grad = True
        trainable_params += p.numel()

print(f"Total parameters count: {total_params}") # ~108M
print(f"Trainable parameters count: {trainable_params}") # ~3k

Total parameters count: 108313348
Trainable parameters count: 3076


In [9]:
LABEL_LIST = ['World','Sports','Business','Sci/Tech'] 
MAX_SEQ_LENGHT = 128


def _create_examples(df, set_type):
    examples = []
    for index, row in df.iterrows():
        guid = f"{index}-{set_type}"
        examples.append(
            InputExample(guid=guid, text_a=row['Description'], label=row['Class Index']))
    return examples

def _df_to_features(df, set_type):
    examples = _create_examples(df, set_type)
    
    # Added for error on backward compatibility with older transformers versions
    legacy_kwards = {}
    from packaging import version
    if version.parse(transformers.__version__) < version.parse("2.9.0"):
        legacy_kwards = {
            "pad_on_left": False,
            "pad_token": tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            "pad_token_segment_id": 0,
        }
    
    return glue_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        label_list=LABEL_LIST,
        max_length=MAX_SEQ_LENGHT,
        output_mode="classification",
        **legacy_kwards,
    )

def _features_to_dataset(features):
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features], dtype=torch.long
    )
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features], dtype=torch.long
    )
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    dataset = TensorDataset(
        all_input_ids, all_attention_mask, all_token_type_ids, all_labels
    )

    return dataset

In [10]:
train_features = _df_to_features(df_train, "train")

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [11]:
test_features = _df_to_features(df_test, "test")

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [21]:
train_dataset = _features_to_dataset(train_features)
test_dataset = _features_to_dataset(test_features)

In [22]:
BATCH_SIZE = 8

In [23]:
train_dataloader = DataLoader(train_dataset)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model = model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, eps=1e-8)

In [25]:
EPOCHS = 2
LOGGING_INTERVAL = 1000

In [26]:

def accuracy(preds, labels):
    return (preds == labels).mean()

def evaluate(model):    
    model.eval()

    loss_arr = []
    accuracy_arr = []
    
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      'labels':         batch[3]}

            outputs = model(**inputs)
            loss, logits = outputs[:2]
            
            preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = inputs['labels'].detach().cpu().numpy()
            
            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))
    
    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

In [27]:
for epoch in range(1, EPOCHS+1):
    losses = []
    for step, batch in enumerate(tqdm(train_dataloader)):
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels':         batch[3]}
        outputs = model(**inputs) # output = loss, logits, hidden_states, attentions
        
        loss = outputs[0]
        loss.backward()
        
        losses.append(loss.item())
        optimizer.step()

        if step > 0 and step % LOGGING_INTERVAL == 0:
            train_loss = np.mean(losses)
            eval_loss, eval_accuracy = evaluate(model)

            print(
                f"Epoch: {epoch} | "
                f"Step: {step} | "
                f"Train loss: {train_loss:.3f} | "
                f"Eval loss: {eval_loss:.3f} | "
                f"Eval accuracy: {eval_accuracy:.3f} | "
            )

Epoch: 1 | Step: 1000 | Train loss: 119.149 | Eval loss: 119.138 | Eval accuracy: 0.250 | 
Epoch: 1 | Step: 2000 | Train loss: 101.884 | Eval loss: 65.927 | Eval accuracy: 0.253 | 
Epoch: 1 | Step: 3000 | Train loss: 92.634 | Eval loss: 58.197 | Eval accuracy: 0.299 | 
Epoch: 1 | Step: 4000 | Train loss: 88.843 | Eval loss: 68.364 | Eval accuracy: 0.316 | 
Epoch: 1 | Step: 5000 | Train loss: 82.924 | Eval loss: 78.208 | Eval accuracy: 0.250 | 
Epoch: 1 | Step: 6000 | Train loss: 78.973 | Eval loss: 56.760 | Eval accuracy: 0.252 | 
Epoch: 1 | Step: 7000 | Train loss: 75.704 | Eval loss: 100.000 | Eval accuracy: 0.336 | 
Epoch: 1 | Step: 8000 | Train loss: 72.840 | Eval loss: 89.826 | Eval accuracy: 0.338 | 
Epoch: 1 | Step: 9000 | Train loss: 71.206 | Eval loss: 52.491 | Eval accuracy: 0.273 | 
Epoch: 1 | Step: 10000 | Train loss: 70.463 | Eval loss: 71.367 | Eval accuracy: 0.279 | 
Epoch: 1 | Step: 11000 | Train loss: 68.400 | Eval loss: 23.043 | Eval accuracy: 0.501 | 
Epoch: 1 | Step

Epoch: 2 | Step: 1000 | Train loss: 66.059 | Eval loss: 72.582 | Eval accuracy: 0.606 | 
Epoch: 2 | Step: 2000 | Train loss: 66.153 | Eval loss: 40.807 | Eval accuracy: 0.634 | 
Epoch: 2 | Step: 3000 | Train loss: 62.101 | Eval loss: 26.786 | Eval accuracy: 0.671 | 
Epoch: 2 | Step: 4000 | Train loss: 58.153 | Eval loss: 37.605 | Eval accuracy: 0.657 | 
Epoch: 2 | Step: 5000 | Train loss: 59.120 | Eval loss: 30.826 | Eval accuracy: 0.668 | 
Epoch: 2 | Step: 6000 | Train loss: 57.679 | Eval loss: 38.732 | Eval accuracy: 0.618 | 
Epoch: 2 | Step: 7000 | Train loss: 55.865 | Eval loss: 19.952 | Eval accuracy: 0.757 | 
Epoch: 2 | Step: 8000 | Train loss: 54.797 | Eval loss: 19.001 | Eval accuracy: 0.756 | 
Epoch: 2 | Step: 9000 | Train loss: 54.254 | Eval loss: 20.756 | Eval accuracy: 0.743 | 
Epoch: 2 | Step: 10000 | Train loss: 53.708 | Eval loss: 22.055 | Eval accuracy: 0.743 | 
Epoch: 2 | Step: 11000 | Train loss: 52.633 | Eval loss: 22.701 | Eval accuracy: 0.731 | 
Epoch: 2 | Step: 12